In [102]:
#Import Modules

#GPyOpt - Cases are important, for some reason
import GPyOpt
from GPyOpt.methods import BayesianOptimization

#numpy
import numpy as np
from numpy.random import multivariate_normal #For later example

import pandas as pd

#Plotting tools
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from numpy.random import multivariate_normal

# --- Load GPyOpt
from GPyOpt.methods import BayesianOptimization
%pylab inline
import GPyOpt
import GPy
import numpy as np
import pickle

Populating the interactive namespace from numpy and matplotlib


In [103]:

def f(x):
    """
    x is a 4D vector.
    Function which will send alpha_1, alpha_2, alpha_3 and alpha_4 
    to the actual model and will get the dice coefficient in return.
    """
    print(x)
    alpha_1 = x[:, 0][0]
    alpha_2 = x[:, 1][0]
    alpha_3 = x[:, 2][0]
    alpha_4 = x[:, 3][0]
    print("--------",alpha_1," ",alpha_2," ",alpha_3," ",alpha_4)
    print(alpha_1+alpha_2+alpha_3+alpha_4)
    # Here we will send the alphas to the actual model and in return 
    # we will recieve the dice coefficient to optimise, since this is
    # a maximization problem, we return the -ve of objective function
    # to be maximized
    dice_coef = alpha_1 + alpha_2 + alpha_3 + alpha_4
    return -dice_coef
    

domain = [{'name': 'alpha_1', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_2', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_3', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_4', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1}]

constraints = [{'name': 'constr_1', 'constraint': '0.9999 - x[:,0] - x[:,1] - x[:,2] - x[:,3]'},
               {'name': 'constr_1', 'constraint': '-1.00001 + x[:,0] + x[:,1] + x[:,2] + x[:,3]'}]

In [104]:
def load_entire_file_into_memory_and_then_convert(filename):
    with open(filename, 'r') as input_file:
        full_file_contents = input_file.read()
        lines_of_file = full_file_contents.split('\n')
        return numpy.array(lines_of_file)

In [105]:
dump = load_entire_file_into_memory_and_then_convert('store_alphas_dice.txt')
dump

array(['0.27 0.02 0.24 0.46 0.9591403057086999 ',
       '0.46 0.07 0.45 0.01 0.961725938544216 ',
       '0.01 0.68 0.11 0.19 0.9603536723291805 ',
       '0.16 0.1 0.5 0.23 0.9574356292382331 ',
       '0.3 0.07 0.32 0.3 0.96022589174643 ',
       '0.56 0.11 0.17 0.15 0.9410618600506602 ',
       '0.05 0.35 0.44 0.15 0.9634017507224188 ',
       '0.22 0.08 0.38 0.31 0.9628548115630068 ',
       '0.03 0.0 0.66 0.3 0.9622572933875356 ',
       '0.14 0.72 0.07 0.06 0.9585863531990674 ',
       '0.11 0.15 0.45 0.28 0.9609590147859615 ',
       '0.16 0.09 0.67 0.07 0.9597778140555175 ',
       '0.71 0.09 0.16 0.03 0.9400787993283924 ',
       '0.47 0.15 0.18 0.19 0.9410591567452614 ',
       '0.17 0.02 0.5 0.3 0.9610975710221837 ',
       '0.34 0.16 0.48 0.01 0.9576296326477832 ',
       '0.06 0.76 0.14 0.03 0.9586400182737919 ',
       '0.36 0.37 0.18 0.09 0.9576236969841502 ',
       '0.42 0.19 0.31 0.08 0.9569187548826379 ',
       '0.25 0.47 0.18 0.1 0.9580239398104423 ', ''], dtype='

In [106]:
X = []
Y = []
for item in dump[:-1]:
    all_items = item.split(' ')
    X.append([float(i) for i in all_items[:4]]) 
    Y.append(float(all_items[-2]))
    

In [107]:
X = numpy.array(X)
X

array([[0.27, 0.02, 0.24, 0.46],
       [0.46, 0.07, 0.45, 0.01],
       [0.01, 0.68, 0.11, 0.19],
       [0.16, 0.1 , 0.5 , 0.23],
       [0.3 , 0.07, 0.32, 0.3 ],
       [0.56, 0.11, 0.17, 0.15],
       [0.05, 0.35, 0.44, 0.15],
       [0.22, 0.08, 0.38, 0.31],
       [0.03, 0.  , 0.66, 0.3 ],
       [0.14, 0.72, 0.07, 0.06],
       [0.11, 0.15, 0.45, 0.28],
       [0.16, 0.09, 0.67, 0.07],
       [0.71, 0.09, 0.16, 0.03],
       [0.47, 0.15, 0.18, 0.19],
       [0.17, 0.02, 0.5 , 0.3 ],
       [0.34, 0.16, 0.48, 0.01],
       [0.06, 0.76, 0.14, 0.03],
       [0.36, 0.37, 0.18, 0.09],
       [0.42, 0.19, 0.31, 0.08],
       [0.25, 0.47, 0.18, 0.1 ]])

In [108]:
X.shape

(20, 4)

In [109]:
Y = -numpy.array(Y)
Y

array([-0.95914031, -0.96172594, -0.96035367, -0.95743563, -0.96022589,
       -0.94106186, -0.96340175, -0.96285481, -0.96225729, -0.95858635,
       -0.96095901, -0.95977781, -0.9400788 , -0.94105916, -0.96109757,
       -0.95762963, -0.95864002, -0.9576237 , -0.95691875, -0.95802394])

In [112]:
Y.shape

(20, 1)

In [111]:
Y = np.expand_dims(Y, axis=1)
Y

array([[-0.95914031],
       [-0.96172594],
       [-0.96035367],
       [-0.95743563],
       [-0.96022589],
       [-0.94106186],
       [-0.96340175],
       [-0.96285481],
       [-0.96225729],
       [-0.95858635],
       [-0.96095901],
       [-0.95977781],
       [-0.9400788 ],
       [-0.94105916],
       [-0.96109757],
       [-0.95762963],
       [-0.95864002],
       [-0.9576237 ],
       [-0.95691875],
       [-0.95802394]])

In [ ]:
Y.shape

In [113]:
maxiter = 1

kernel = GPy.kern.Matern52(input_dim=4, ARD=True, variance=1, lengthscale=[1,1,1,1]);

myBopt_4d = GPyOpt.methods.BayesianOptimization(f=f, X=X, Y=Y, domain=domain, constraints = constraints,
                                                kernel=kernel, acquisition_type ='EI', model_type='GP', 
                                                verbosity=True, acquisition_optimizer_type='lbfgs', 
                                                cost_withGradients=None, exact_feval=True)

myBopt_4d.run_optimization(max_iter = maxiter, verbosity=True)
print("="*20)
print("Value of (x,y) that minimises the objective:"+str(myBopt_4d.x_opt))    
print("Minimum value of the objective: "+str(myBopt_4d.fx_opt))     
print("="*20)
#myBopt_4d.plot_acquisition()

[[0.23070297 0.00674209 0.39912914 0.36343547]]
-------- 0.23070296550728328   0.006742091006239273   0.39912914455401327   0.36343547111957786
1.0000096721871137
num acquisition: 1, time elapsed: 139.79s
Value of (x,y) that minimises the objective:[0.23070297 0.00674209 0.39912914 0.36343547]
Minimum value of the objective: -1.0000096721871137


In [88]:
myBopt_4d.X.shape

(6, 4)

In [86]:
myBopt_4d.Y

array([[-0.99990714],
       [-0.99998241],
       [-0.99992333],
       [-0.99994381],
       [-1.00000402],
       [-0.99999817]])